[View in Colaboratory](https://colab.research.google.com/github/AhmadHAW/ChatBot/blob/develop/training_test.ipynb)

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
from google.colab import files
from google.colab import drive
from os import walk
from pandas import DataFrame
from struct import unpack

from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [0]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
data_dict = {'label': [], 'in_phase': [], 'quadrature_phase': []}

for subdirs, dirs, files in walk("/content/gdrive/My Drive/HAW/Project Lernende Agenten/samples/iq"):
  count = 0
  out = display(progress(0, len(files)), display_id=True)
  for file in files:
    label = file[0:2]
    with open("/content/gdrive/My Drive/HAW/Project Lernende Agenten/samples/iq/{}".format(file), 'rb') as f:
        buffer = f.read()
        num_floats = len(buffer)//4
        floats = unpack("f"*num_floats, buffer)
        i = floats[0::2]
        q = floats[1::2]
        data_dict['label'].append(label)
        data_dict['in_phase'].append(i)
        data_dict['quadrature_phase'].append(q)
    count += 1
    out.update(progress(count, len(files)))
df = DataFrame(data=data_dict)

In [0]:
training_set_size = 180 #@param {type:"slider", min:1, max:3000, step:1}
validating_set_size = 20 #@param {type:"slider", min:1, max:1000, step:1}
test_set_size = 20 #@param {type:"slider", min:1, max:1000, step:1}
learning_rate = 0.3 #@param ["3", "1", "0.3", "0.1", "0.03", "0.01", "0.003", "0.001", "0.0003", "0.0001"] {type:"raw"}
activation_function = "RELU" #@param ["RELU", "Sigmoid", "Tanh"]
regression = "None" #@param ["None", "L1", "L2"]
regression_rate = 3 #@param ["3", "1", "0.3", "0.1", "0.03", "0.01", "0.003", "0.001"] {type:"raw"}
batch_size = 10 #@param {type:"slider", min:1, max:100, step:1}
steps = 500 #@param {type:"slider", min:1, max:2000, step:1}
model = [3,1,5]

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format


In [0]:
df = df.reindex(np.random.permutation(df.index))

training_examples = df.head(training_set_size)["in_phase"]
training_targets = df.head(training_set_size)["label"]

validation_examples = df.tail(validating_set_size)["in_phase"]
validation_targets = df.tail(validating_set_size)["label"]


# Double-check that we've done the right thing.
print("Training examples summary:")
display(training_examples.describe())
print("Validation examples summary:")
display(validation_examples.describe())

print("Training targets summary:")
display(training_targets.describe())
print("Validation targets summary:")
display(validation_targets.describe())

Training examples summary:


count                                                   180
unique                                                  180
top       (0.12009622901678085, -0.9784194231033325, 0.6...
freq                                                      1
Name: in_phase, dtype: object

Validation examples summary:


count                                                    20
unique                                                   20
top       (1.0192893743515015, 1.0026243925094604, 1.003...
freq                                                      1
Name: in_phase, dtype: object

Training targets summary:


count     180
unique      2
top        am
freq       94
Name: label, dtype: object

Validation targets summary:


count     20
unique     2
top       fm
freq      12
Name: label, dtype: object

In [0]:
def preprocess_features(dataframe):
  """Prepares input features from the GNU Radio Signal data set.

  Args:
    dataframe: A Pandas DataFrame expected to contain data
      from the GNU Radio Signal data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """
  selected_features = dataframe.copy()
  selected_features = selected_features[1:len(selected_features.columns)]
  return selected_features

def preprocess_targets(dataframe):
  """Prepares target features (i.e., labels) from GNU Radio Signal data set.

  Args:
    dataframe: A Pandas DataFrame expected to contain data
      from the GNU Radio Signal data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  # Scale the target to be in units of thousands of dollars.
  output_targets["label"] = (
    dataframe["label"])
  return output_targets

In [0]:
training_examples = preprocess_features(dataframe.head(training_set_size))
training_examples.describe()

In [0]:
training_targets = preprocess_targets(dataframe.head(training_set_size))
training_targets.describe()

In [0]:
validation_examples = preprocess_features(dataframe.tail(validating_set_size))
validation_examples.describe()

In [0]:
validation_targets = preprocess_targets(dataframe.tail(validating_set_size))
validation_targets.describe()

In [0]:
def train_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
  """Trains a linear regression model.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    training_examples: A `DataFrame` containing one or more columns from
      `GNU Radio Signal data set` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `GNU Radio Signal data set` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `GNU Radio Signal data set` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `GNU Radio Signal data set` to use as target for validation.
      
  Returns:
    A `LinearRegressor` object trained on the training data.
  """

  periods = 10
  steps_per_period = steps / periods

  # Create a linear regressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=construct_feature_columns(training_examples),
      optimizer=my_optimizer
  )
    
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["label"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["label"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["label"], 
                                                    num_epochs=1, 
                                                    shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period,
    )
    # Take a break and compute predictions.
    training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = linear_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print("Model training finished.")

  
  # Output a graph of loss metrics over periods.
  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  return linear_regressor

In [0]:
train_model(
    learning_rate=learning_rate,
    steps=steps,
    batch_size=batch_size,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)